# Covariance Estimation

In [1]:
# used during development to releoad modules every time there is a change
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize
import statsmodels.api as sm
from course_1.risk_kit import Metrics, pre_processing_hfi, \
pre_processing_ind, get_ind30_nfirms, \
get_ind30_size, plot_ef2, get_fff_returns, \
get_ind_market_caps, get_ind30_nfirms, weight_ew, weight_cw, \
weight_ew_cap_teth

from data_prep import get_ind_returns, get_ind_returns2, get_ind_market_caps

import ipywidgets as widgets
import ipywidgets as widgets
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.float_format = '{:.6f}'.format

m= Metrics()